<a href="https://colab.research.google.com/github/Abbhi1234/AI/blob/main/week5_SimulatedAnnealing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from datetime import datetime
import random, time, math
import decimal

class Board:
    def __init__(self, queen_count=8, queens=None):
        self.queen_count = queen_count
        if queens:
            self.queens = queens
        else:
            self.reset()

    def reset(self):
        # Randomly reset board
        self.queens = [random.randint(0, self.queen_count - 1) for _ in range(self.queen_count)]

    def calculateCost(self):
        threat = 0
        for queen in range(0, self.queen_count):
            for next_queen in range(queen + 1, self.queen_count):
                if self.queens[queen] == self.queens[next_queen] or abs(queen - next_queen) == abs(self.queens[queen] - self.queens[next_queen]):
                    threat += 1
        return threat

    @staticmethod
    def calculateCostWithQueens(queens):
        threat = 0
        queen_count = len(queens)
        for queen in range(0, queen_count):
            for next_queen in range(queen + 1, queen_count):
                if queens[queen] == queens[next_queen] or abs(queen - next_queen) == abs(queens[queen] - queens[next_queen]):
                    threat += 1
        return threat

    @staticmethod
    def toString(queens):
        board_string = ""
        for row, col in enumerate(queens):
            board_string += "(%s, %s)\n" % (row, col)
        return board_string

    def __str__(self):
        board_string = ""
        for row, col in enumerate(self.queens):
            board_string += "(%s, %s)\n" % (row, col)
        return board_string


class SimulatedAnnealing:
    def __init__(self, board):
        self.elapsedTime = 0
        self.board = board
        self.temperature = 1000
        self.sch = 0.995
        self.startTime = datetime.now()
        self.max_iterations = 10000

    def run(self):
        board = self.board
        board_queens = self.board.queens[:]
        solutionFound = False

        for k in range(0, self.max_iterations):
            self.temperature *= self.sch
            successor_queens = board_queens[:]

            queen_to_move = random.randint(0, self.board.queen_count - 1)
            new_position = random.randint(0, self.board.queen_count - 1)


            while new_position == successor_queens[queen_to_move]:
                new_position = random.randint(0, self.board.queen_count - 1)

            successor_queens[queen_to_move] = new_position

            dw = Board.calculateCostWithQueens(successor_queens) - Board.calculateCostWithQueens(board_queens)
            exp = decimal.Decimal(math.e) ** (decimal.Decimal(-dw) * decimal.Decimal(self.temperature))


            if dw > 0 or random.uniform(0, 1) < exp:
                board_queens = successor_queens[:]

            if Board.calculateCostWithQueens(board_queens) == 0:
                print("Solution found:")
                print(Board.toString(board_queens))
                self.elapsedTime = self.getElapsedTime()
                print("Success, Elapsed Time: %sms" % (str(self.elapsedTime)))
                solutionFound = True
                break

        if not solutionFound:
            self.elapsedTime = self.getElapsedTime()
            print("Unsuccessful, Elapsed Time: %sms" % (str(self.elapsedTime)))

        return self.elapsedTime

    def getElapsedTime(self):
        endTime = datetime.now()
        elapsedTime = (endTime - self.startTime).microseconds / 1000
        return elapsedTime


def get_initial_queens(queen_count):
    queens = []
    print(f"Please enter the initial positions of {queen_count} queens:")
    for i in range(queen_count):
        while True:
            try:
                col = int(input(f"Enter the column position for queen {i+1} (0 to {queen_count-1}): "))
                if col < 0 or col >= queen_count:
                    raise ValueError("Column out of range.")
                queens.append(col)
                break
            except ValueError as e:
                print(e)
    return queens


if __name__ == '__main__':
    queen_count = int(input("Enter the number of queens (default is 8): ") or 8)
    queens = get_initial_queens(queen_count)

    board = Board(queen_count, queens)
    print("Initial Board:")
    print(board)

    SimulatedAnnealing(board).run()



Enter the number of queens (default is 8): 4
Please enter the initial positions of 4 queens:
Enter the column position for queen 1 (0 to 3): 2
Enter the column position for queen 2 (0 to 3): 1
Enter the column position for queen 3 (0 to 3): 0
Enter the column position for queen 4 (0 to 3): 3
Initial Board:
(0, 2)
(1, 1)
(2, 0)
(3, 3)

Solution found:
(0, 2)
(1, 0)
(2, 3)
(3, 1)

Success, Elapsed Time: 20.612ms


In [5]:
import numpy as np
import random
import math

def queens_cost(queens):
    n = len(queens)
    cost = 0
    for i in range(n):
        for j in range(i + 1, n):
            if queens[i] == queens[j]:
                cost += 1
            elif abs(queens[i] - queens[j]) == abs(i - j):
                cost += 1
    return cost

def simulated_annealing(n, max_iters=5000, initial_temp=1000, cooling_rate=0.995, initial_state=None):
    if initial_state is None:
        queens = np.random.randint(0, n, size=n)
    else:
        queens = np.array(initial_state)

    current_cost = queens_cost(queens)
    temp = initial_temp

    best_state = queens.copy()
    best_cost = current_cost

    for _ in range(max_iters):
        neighbor = queens.copy()
        i = random.randint(0, n - 1)
        neighbor[i] = random.randint(0, n - 1)

        neighbor_cost = queens_cost(neighbor)

        if neighbor_cost < current_cost:
            queens = neighbor
            current_cost = neighbor_cost
        else:
            if random.random() < math.exp(-(neighbor_cost - current_cost) / temp):
                queens = neighbor
                current_cost = neighbor_cost

        if current_cost < best_cost:
            best_state = queens.copy()
            best_cost = current_cost

        temp *= cooling_rate

    return best_state, best_cost

def get_input():
    n = int(input("Enter the number of queens: "))
    initial_state = []
    print(f"Enter the column positions for each queen (values between 0 and {n-1}):")
    for i in range(n):
        while True:
            try:
                col = int(input(f"Enter column for queen {i + 1}: "))
                if col < 0 or col >= n:
                    raise ValueError(f"Column must be between 0 and {n - 1}.")
                initial_state.append(col)
                break
            except ValueError as e:
                print(e)
    return n, initial_state

n, initial_state = get_input()

best_solution, best_cost = simulated_annealing(n, initial_state=initial_state)

print(f"The best solution found is: {best_solution}")
print(f"The number of attacking queen pairs is: {best_cost}")

for i, col in enumerate(best_solution):
    print(f"Queen {i+1} is at position ({i}, {col})")






Enter the number of queens: 8
Enter the column positions for each queen (values between 0 and 7):
Enter column for queen 1: 0
Enter column for queen 2: 5
Enter column for queen 3: 6
Enter column for queen 4: 7
Enter column for queen 5: 2
Enter column for queen 6: 1
Enter column for queen 7: 3
Enter column for queen 8: 4
The best solution found is: [6 4 2 0 5 7 1 3]
The number of attacking queen pairs is: 0
Queen 1 is at position (0, 6)
Queen 2 is at position (1, 4)
Queen 3 is at position (2, 2)
Queen 4 is at position (3, 0)
Queen 5 is at position (4, 5)
Queen 6 is at position (5, 7)
Queen 7 is at position (6, 1)
Queen 8 is at position (7, 3)
